# DES
- DES是使用最广泛的**对称密码**：数据加密标准（Data Encryption Standard）。<br>
- 在对称密码系统（也叫做**私钥密码系统**）中，加密和解密使用**相同的密钥**。<br>
<div>
    <img align=left src="Images/des.png"/>
</div>

## 1. DES原理
### 1.1 基本工作原理
用56位的密钥对64位长的数据块进行16轮加密处理由此得到64位长的密文。
### 1.2 基本工作流程
<div>
    <img align=left src="Images/des-encryption-process.png"/>
</div>